# TRABALHO FINAL DE MATEMÁTICA DISCRETA

## PARTICIPANTES

- Leonardo Alexandre da Silva Ferreira - 2º Período - Ciência de Dados - 231708030;
- Matheus Fillype Ferreira de Carvalho - 2º Período - Ciência de Dados - 231708017;
- Sillas Rocha da Costa - 2º Período - Ciência de Dados - 231708014.

## INTRODUÇÃO:

O trabalho tem como objetivo aplicar a teoria de grafos à dinâmica de aeroportos, conforme foi mencionado no relatório inicial que foi enviado. São dois os principais objetivos do projeto: o primeiro é criar um programa que possa desenvolver rotas aéreas complexas usando o algoritmo de Dijkstra; isto é, rotas que incluam mais de uma parada obrigatória. Dessa forma, o programa retornaria todo o 'plano de voo'. O segundo objetivo é identificar aeroportos cruciais, ou seja, aeroportos cuja remoção resultaria no maior número de componentes. Na prática, isso significaria encontrar os aeroportos que, caso deixassem de operar por algum motivo, tornariam inviável o acesso a diversos outros aeroportos.

## EXTRAINDO OS DADOS:

Conseguir os dados para executar o projeto foi, de fato, complicado. Procuramos em sites como o [OpenFlights](https://openflights.org/#), que realmente possuem dados sobre as inúmeras linhas aéreas existentes no planeta. No entanto, o grande problema era extrair todos os dados de uma só vez, uma vez que não existia um mecanismo que possibilitasse o download de todas as informações de uma só vez.

A solução encontrada foi uma base de dados presente no site [Kaggle](https://www.kaggle.com/datasets/open-flights/flight-route-database) com dados sobre linhas aéreas entre mais de 3 mil aeroportos em diversos países em janeiro de 2012. Obviamente, desde então, novos aeroportos foram criados e, consequentemente, novas conexões. Contudo, visto que o objetivo do trabalho é aplicar a teoria vista em aula para problemas da vida real, problemas práticos, acredito que a desatualização da base de dados não seja um problema.

Uma vez com os dados em mãos, teríamos que tratá-los para que pudessem atender às nossas necessidades. Abaixo está o código usado para tratar a base de dados inicial e gerar duas bases de dados derivadas: uma apenas com aeroportos brasileiros e outra com todos. Em relação ao tratamento, um comentário relevante é que utilizamos, além da biblioteca `pandas`, a biblioteca de Python `airportsdata` para traduzir os dados, visto que a base inicial possui apenas o código IATA (International Air Transport Association) dos aeroportos, e precisaríamos de outros dados, como latitude e longitude, para calcular as distâncias entre eles, por exemplo.

In [1]:
import pandas as pd  
import airportsdata
import numpy as np 

#lendo o dataframe
dataframe_bruto = pd.read_csv("database/routes.csv")

#carregando dados da biblioteca
dicionario_dos_aeroportos = airportsdata.load('IATA')

#dados que vamos extrai da biblioteca que não existem no dataframe inicial
lista_de_colunas = ["route_id", "source_airport_icao", "source_airport_name", "source_airport_city", "source_airport_country", "source_airport_elevation",
                     "source_airport_lat", "source_airport_lon", "destination_airport_icao", "destination_airport_name", "destination_airport_city", 
                     "destination_airport_country", "destination_airport_elevation", "destination_airport_lat", "destination_airport_lon"]

lista_de_origens = list(dataframe_bruto[" source airport"])
lista_de_destinos = list(dataframe_bruto[" destination apirport"])
lista_de_dados = list()

#extraindo dados da biblioteca
for cada_origem, cada_destino in zip(lista_de_origens, lista_de_destinos):
    try:
        lista_auxiliar = list()
        lista_auxiliar.append(f"{cada_origem}-{cada_destino}")
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["icao"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["name"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["city"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["country"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["elevation"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["lat"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["lon"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["icao"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["name"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["city"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["country"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["elevation"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["lat"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["lon"])
        lista_de_dados.append(lista_auxiliar)
    except:
        lista_de_dados.append(list())

dataframe_filtrado = pd.DataFrame(data=lista_de_dados, columns=lista_de_colunas)
dataframe_bruto["route_id"] = dataframe_filtrado["route_id"]

#criando o dataframe final
dataframe_final = pd.merge(dataframe_bruto, dataframe_filtrado, on='route_id', how='inner')
dataframe_final = dataframe_final.dropna(thresh=2)
dataframe_final = dataframe_final.groupby("route_id", as_index=False).first().reset_index(drop=True)

#filtrando para aeroportos brasileiros
dataframe_final_brasil = dataframe_final[dataframe_final["source_airport_country"] == "BR"].reset_index(drop=True)

""" TRECHO PARA SALVAR AS NOVAS BASES DE DADOS
dataframe_final.to_csv("database\\dataframe_final_mundo.csv")
dataframe_final_brasil.to_csv("database\\dataframe_final_brasil.csv")
"""

print(dataframe_final.shape)

(36800, 24)


## CÁLCULO DE ROTAS:

Como foi exposto no documento enviado com os objetivos do trabalho, um dos nossos objetivos é desenvolver um programa com base no algoritmo de Dijkstra. Esse algoritmo é utilizado para encontrar o menor caminho entre dois vértices em um grafo dirigido, e no nosso caso, será utilizado para encontrar as rotas aéreas mais eficientes com base nas horas de voo.

Obviamente, entre dois vértices, o uso do algoritmo seria relativamente simples. No entanto, a nossa aplicação visa encontrar as rotas mais eficientes em viagens complexas, que envolvem múltiplas paradas. Essas paradas devem estar ordenadas.

No nosso modelo, os aeroportos são representados como vértices, e as rotas aéreas como arestas. Entretanto, em nossa base de dados, não possuímos informações sobre as durações dos voos. Como alternativa, optamos por utilizar latitude e longitude para calcular a distância aproximada entre eles. Posteriormente, calculamos a duração da viagem para um avião comercial médio.

Segue abaixo o código utilizado para calcular a distância entre os aeroportos, levando em consideração a curvatura uniforme da Terra, e a duração aproximada dessas viagens:

In [2]:
import math

def cal_dist(lat_inicial:float, lon_inicial:float, lat_final:float, lon_final:float) -> float:
    """
    """
    try:
        dif_lat = math.radians(float(lat_final) - float(lat_inicial))
        dif_lon = math.radians(float(lon_final) - float(lon_inicial))

        raio_terra_km = 6370

        #funções de haversine para distância esférica
        hav_lat = math.sin(dif_lat/2)**2
        hav_lon = math.sin(dif_lon/2)**2

        cos_1 = math.cos(float(math.radians(lat_inicial)))
        cos_2 = math.cos(float(math.radians(lat_final)))

        dist_real = 2*raio_terra_km*math.asin(math.sqrt(hav_lat + cos_1 * cos_2 * hav_lon))

        return round(dist_real, 3)
    
    except:
        return None


def cal_horas(dist_km:float) -> float:
    """
    """
    try:
        velociade_km_h = 850
        tempo_horas = round(float(dist_km)/velociade_km_h, 3)

        return tempo_horas
    
    except:
        return None

Definidas as funções, vamos agora aplicá-las linha a linha e salvar no nosso dataframe a distância entre os aeroportos em questão, ou seja, o comprimento da rota. Além disso, vamos adicionar ao dataframe a duração estimada da viagem com base na velocidade de um avião comercial médio, como foi mencionado anteriormente. Essa duração será representada no formato decimal, uma vez que esses dados serão utilizados exclusivamente para comparações e operações matemáticas relacionadas à aplicação do algoritmo de Dijkstra.

In [3]:
lista_lat_inicial = list(dataframe_final["source_airport_lat"])
lista_lon_inicial = list(dataframe_final["source_airport_lon"])
lista_lat_final = list(dataframe_final["destination_airport_lat"])
lista_lon_final = list(dataframe_final["destination_airport_lon"])

lista_distancias = list()
lista_tempo_de_viagem = list()
for index in range(0, len(lista_lat_inicial)):

    #extraindo os dados
    lat_inicial = lista_lat_inicial[index]
    lon_inicial = lista_lon_inicial[index]
    lat_final = lista_lat_final[index]
    lon_final = lista_lon_final[index]

    #claculando a distância e o tempo
    dist = cal_dist(lat_inicial, lon_inicial, lat_final, lon_final)
    tempo = cal_horas(dist)

    lista_distancias.append(dist)
    lista_tempo_de_viagem.append(tempo)

#criando as colunas com os novos dados 
dataframe_final["distancia"] = lista_distancias
dataframe_final["tempo_de_viagem"] = lista_tempo_de_viagem

#excluindo linhas, que por algum problema, não foi possível calcular ou a distância, ou o tempo de viagem
dataframe_final = dataframe_final.dropna(thresh=2).reset_index(drop=True)

print(dataframe_final.shape)

(36800, 26)


Após inserir na nossa base de dados as informações necessárias, basta agora criar o programa que recebe o nome de cidades em ordem. Essas cidades devem ter aeroportos, e o programa retorna uma espécie de plano de voo buscando a menor rota entre cada aeroporto. Inicialmente, vamos converter o dataframe para o formato de "grafo", que é um dicionário para cada aeroporto, onde cada chave representa um aeroporto "conectado" a ele e o valor é a distância entre os dois. Abaixo está o código usado para construir o grafo, e no output, um exemplo de como é a estrutura do nosso grafo/dicionário.

In [4]:
grafo_com_pesos = dict()

#criando lista com todos os aeroportos
aeroportos_destino = set(dataframe_final[" destination apirport"])
aeroportos_origem = set(dataframe_final[" source airport"])

#criando os dicionário referente a cada aeroporto
for cada_aeroporto in aeroportos_origem.union(aeroportos_destino):
    grafo_com_pesos[cada_aeroporto] = dict()

#atribuindo nos dicionários de cada aeroporto dos os dicionários adjacentes e suas respectivas distâncias
for cada_aeroporto in aeroportos_destino:
    dataframe_rotas_dest = dataframe_final[dataframe_final[" source airport"] == cada_aeroporto]
    lista_de_destinos = list(dataframe_rotas_dest[" destination apirport"])
    lista_de_distancias = list(dataframe_rotas_dest["distancia"])

    for cada_destino, cada_distancia in zip(lista_de_destinos, lista_de_distancias):
        grafo_com_pesos[cada_aeroporto][cada_destino] = cada_distancia

for cada_aeroporto in aeroportos_origem:
    dataframe_rotas_ori = dataframe_final[dataframe_final[" destination apirport"] == cada_aeroporto]
    lista_de_destinos = list(dataframe_rotas_ori[" source airport"])
    lista_de_distancias = list(dataframe_rotas_ori["distancia"])

    for cada_destino, cada_distancia in zip(lista_de_destinos, lista_de_distancias):
        grafo_com_pesos[cada_aeroporto][cada_destino] = cada_distancia

#deletando eventuais dicionários vazios
grafo_final = grafo_com_pesos.copy()
"""
for cada_chave, cada_aeroporto in grafo_com_pesos.items():
    if len(cada_aeroporto) == 0:
        del grafo_final[cada_chave]
"""
print(grafo_final["STZ"])

{}


Após criar o grafo com os aeroportos, o próximo passo foi implementar o algorítimo de Dijkstra na linguagem pytho, para que fosse possível encontrar o menor trajeto entre dois aeroportos. Segue abaixo o código referente à implementação e um exemplo do retorno da função. A tupla retornada contém, no primeiro campo, um string com as siglas dos aeroportos presentes no menor caminho e, no segundo campo, a distância total do trajeto. 

In [5]:
def dijkstra(origem:str, destino:str) -> tuple: 
    """
    """
    
    grafo_genérico = grafo_final

    dict_caminhos = dict()

    dict_distancias = dict()
    for cada_chave in grafo_genérico.keys():
        dict_distancias[cada_chave] = np.inf
        dict_caminhos[cada_chave] = ""

    lista_vertices_visitados = list()

    dict_distancias[origem] = 0
    vertice_da_vez = origem

    while True:
        lista_vertices_visitados.append(vertice_da_vez)

        for cada_conexao_da_vez, cada_peso_da_vez in grafo_genérico[vertice_da_vez].items():
            if cada_conexao_da_vez in lista_vertices_visitados:
                continue
            else:
                if dict_distancias[cada_conexao_da_vez] > cada_peso_da_vez + dict_distancias[vertice_da_vez]:
                    dict_distancias[cada_conexao_da_vez] = cada_peso_da_vez + dict_distancias[vertice_da_vez]
                    dict_caminhos[cada_conexao_da_vez] = dict_caminhos[vertice_da_vez] + "-" + vertice_da_vez


        menor_dist = np.inf
        for cada_chave in set(dict_distancias.keys()).union(set(lista_vertices_visitados)) - set(dict_distancias.keys()).intersection(set(lista_vertices_visitados)):
            if dict_distancias[cada_chave] < menor_dist:
                menor_dist = dict_distancias[cada_chave]
                vertice_da_vez = cada_chave

        if vertice_da_vez == destino:
            dict_caminhos[destino] = dict_caminhos[destino] + "-" + destino + "-"
            break

    return dict_caminhos[destino], dict_distancias[destino]

print(dijkstra("CNF", "DUB"))

('-CNF-LIS-DUB-', 9080.153)


Após implementar o algorítimo, seria necessário aplicá-lo à dinâmica de vigens complexas. Na célula abaixo, além da função que retorna o plano de voo de uma viagem com multiplas paradas obrigatórias, existe um exemplo de como ela funciona na prática. 

Imagine que seja necessário que alguém, por algum motivo, viaje entre as cidades de São Paulo, Dubai, Miami, Belo Horizonte e, em seguida, retorne para a cidade de São Paulo. Todas as paradas citadas acima estão ordenadas, ou seja, o viajante deve visitá-las em ordem. 

O programa abaixo, utiliza da função `Dijkstra`, desenvolvida acima, para encontrar o melhor, e mais eficiente, plano de voo para realizar a viagem em questão considerando a existência de linhas aéreas entre as cidades.

Inicialmente, ele partiria do aeroporto de `São Paulo-BRA (GRU)` para o aeroporto de `Porto-POR (OPO)` para, finalmente, chegar no aeroporto de `Dubai-EAU (DUB)`. Em seguida, para ir de Dubai para Miami, ele teria que fazer uma parada no aeroporto de `Boston-EUA (BOS)` para, em seguida, chegar no aeroporto de `Miami-EUA (MIA)`. Por fim, como existem voos diretos entre Miami-Belo Horizonte e entre Belo Horizonte-São Paulo, o trajeto mais eficiente é embarcar em voos diretos e finalizar a viagem. 

In [6]:
lista_geral = ["GRU", "DUB", "MIA", "CNF"]

def rota_completa(lista_de_destinos:str) -> tuple:
    """
    """
    distancia_total = 0
    caminho_total = ""

    for indice in range(0, len(lista_de_destinos)):
        if indice == len(lista_de_destinos) - 1:
            caminho_auxiliar, distancia_auxiliar = dijkstra(lista_de_destinos[indice], lista_de_destinos[0])
            caminho_total += caminho_auxiliar
            distancia_total += distancia_auxiliar

        else:
            caminho_auxiliar, distancia_auxiliar = dijkstra(lista_de_destinos[indice], lista_de_destinos[indice + 1])
            caminho_total += caminho_auxiliar
            distancia_total += distancia_auxiliar

    return caminho_total, round(distancia_total, 3)

print(rota_completa(lista_geral))

('-GRU-OPO-DUB--DUB-BOS-MIA--MIA-CNF--CNF-GRU-', 23264.096)


Vamos pensar em um exemplo mais complexo, imagine que um milionário queira acompanhar todo o campeonato mundial de fórmula 1. Claramente, ele terá que viajar para diversos países ao longo do ano em seu avião particular e encontrar o plano de voo mais eficiente significaria uma economia considerável de horas de viagem. No output, temos todos os planos de voo que o milionário deve seguir, obviamente, se o avião particular tiver autonomia para realizar um voo direto, o ideal é que isso seja feito, mas estamos considerando um cenário onde só é possível viajar pelas rotas já existentes e que o tempo gasto para uma eventual parada é desconsiderado, logo, não existe problema em realizar inúmeras escalas, desde que o trajeto seja o menor possível.

In [7]:
dicionario_f1 = {"Rio de Janeiro":"GIG",
                 "Bahrain":"BAH",
                 "Riyadh-Arábia Saudita":"SAW",
                 "Melbourne-Austrália":"MEL",
                 "Baku-Azerbaijão":"GYD",
                 "Miami-EUA":"MIA",
                 "Doha-Qatar":"DOH",
                 "Mônaco (Aeroporto de Nice-FRA)":"NCE",
                 "Barcelona-Espanha":"BCN",
                 "Montreal-Canadá":"YUL",
                 "Spielberg (Viena-AUS)":"VIE",
                 "Silverstone (Manchester-ING)":"MAN",
                 "Budapeste-Hungria":"BUD",
                 "Stavelot (Bruxelas-BEL)":"CRL",
                 "Zandvoort (Amsterdã-HOL)":"AMS",
                 "Monza (Milão-ITA)":"MXP",
                 "Singapura":"SIN",
                 "Suzuka (Tokio-JAP)":"NRT",
                 "Lusail (Doha-QAT)":"DOH",
                 "Austin-EUA":"AUS",
                 "Cidade do México-México":"MEX",
                 "São Paulo-Brasil":"GRU",
                 "Las Vegas-EUA":"LAS",
                 "Abu Dhabi-EAU":"AUH",
}

lista_de_plano = rota_completa(list(dicionario_f1.values()))[0].split("--")
for indice in range(len(lista_de_plano)):
    print(f"Voo {indice + 1} : {lista_de_plano[indice]}")

Voo 1 : -GIG-LFW-ABV-NDJ-JED-BAH
Voo 2 : BAH-KWI-SAW
Voo 3 : SAW-DXB-MEL
Voo 4 : MEL-DEL-ASB-GYD
Voo 5 : GYD-KBP-TXL-MIA
Voo 6 : MIA-ZRH-DOH
Voo 7 : DOH-FCO-NCE
Voo 8 : NCE-BCN
Voo 9 : BCN-YUL
Voo 10 : YUL-AMS-VIE
Voo 11 : VIE-MAN
Voo 12 : MAN-BUD
Voo 13 : BUD-CRL
Voo 14 : CRL-MAN-AMS
Voo 15 : AMS-MXP
Voo 16 : MXP-SIN
Voo 17 : SIN-NRT
Voo 18 : NRT-DOH
Voo 19 : DOH-ORD-AUS
Voo 20 : AUS-MEX
Voo 21 : MEX-GRU
Voo 22 : GRU-PTY-LAS
Voo 23 : LAS-DEN-KEF-OSL-VNO-MSQ-AUH
Voo 24 : AUH-JED-NDJ-ABV-LFW-GIG-


Agora, elabora-se um top 11 dos aeroportos de destino conectados com aeroportos de origem que possuem voos com apenas 1 opção de destino. Ou seja, retorna-se um top 11 dos aeroportos que contém maiores números de conexões com aeroportos de voos únicos (1 opção de destino). Nesse caso, ao considerar os aeroportos como vértices, recebe-se um top 11 de vértices (aeroportos) que possuem maiores quantidades de conexões com vértices (aeroportos) de grau 1. Assim, obtém-se os vértices (aeroportos) que, ao serem retirados, geram mais componentes conexas no grafo.

In [8]:
contagem = dataframe_final['source_airport_name'].value_counts()
dataframe = dataframe_final[dataframe_final['source_airport_name'].map(contagem) == 1].reset_index(drop=True)
dataframe_filtrado = dataframe['destination_airport_name'].value_counts().head(11)
print(dataframe_filtrado)

destination_airport_name
Tokyo International Airport                       14
El Dorado International Airport                   14
Denver International Airport                      14
Jorge Newbery Airpark                             13
Ninoy Aquino International Airport                13
Don Mueang International Airport                  12
Jorge Chavez International Airport                11
Mehrabad International Airport                    11
Dallas-Fort Worth International Airport           11
Domodedovo International Airport                  11
Licenciado Benito Juarez International Airport    11
Name: count, dtype: int64



Por outro lado, retorna-se um top 10 dos aeroportos que mais possuem registros na coluna "destination_airport_name". Em outras palavras, obtém-se um top 10 dos aeroportos que mais podem receber voos de aeroportos distintos.

In [9]:
top_10_destino = dataframe_final["destination_airport_name"].value_counts().head(10)
print(top_10_destino)

destination_airport_name
Frankfurt am Main International Airport               237
Charles de Gaulle International Airport               233
Amsterdam Airport Schiphol                            230
Istanbul Airport                                      225
Hartsfield - Jackson Atlanta International Airport    216
Chicago O'Hare International Airport                  203
Beijing Capital International Airport                 198
Munich International Airport                          189
Domodedovo International Airport                      187
Dallas-Fort Worth International Airport               185
Name: count, dtype: int64


## CONCLUSÃO:

Neste trabalho, a aplicação da teoria de grafos à dinâmica de aeroportos permitiu a criação de um programa capaz de desenvolver rotas aéreas complexas usando o algoritmo de Dijkstra. O foco do programa é encontrar a rota mais eficiente, em relação a distância, entre dois aeroportos, através, apenas, de aeroportos que possuem voos diretos entre si.

Na base de dados incial, apenas constava as posições relativas a longitude e latitude dos aeroportos, por isso, foi necessário utilizar a Fórmula de Haversine, equação usada em navegação, fornecendo distâncias entre dois pontos de uma esfera a partir de suas latitudes e longitudes.

Para realizar a análise, foi criado um grafo, simples e com pesos de distâncias, representando os aeroportos e suas conexões, utilizado como base para o algoritmo de Dijkstra. O programa foi testado com sucesso em diferentes cenários, fornecendo resultados coerentes e úteis. Isso é facilmente visto no exemplo anterior sobre os GPs de Fórmula 1.

Deste modo, foi possível chegar a conclusões como, a identificação de aeroportos cruciais cuja remoção afetaria significativamente a conectividade da rede, para isto, foi necessário verificar os aeroportos que possuiam mais aeroportos de grau 1 conectados a eles, em que Aeroporto Internacional de Tóquio (Japão), Aeroporto Internacional El Dorado (Colômbia) e Aeroporto Internacional de Denver (Estados Unidos), com 14 aeroportos de grau 1 conectados a eles. Adicionalmente, foram apresentados dados estatísticos sobre a frequência de destinos nos dados coletados, destacando os aeroportos mais populares, aeroportos com mais conexão, ou seja, maior grau, onde, em primeiro lugar, o Aeroporto de Frankfurt am Main (Alemanha), que possui 237 conexões, em segundo, o Aeroporto de Paris-Charles de Gaulle (França), que possui 233 conexões e, em terceiro lugar, o Aeroporto de Amesterdão-Schiphol (Países Baixos), que possui 230 conexões. Resultando em um melhor entendimento sobre as dinâmicas aéreas.